<a href="https://colab.research.google.com/github/CodeHunterOfficial/ABC_DataMining/blob/main/NM/Stable_Diffusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Установка необходимых библиотек
# !pip install diffusers transformers accelerate

from diffusers import DiffusionPipeline
import torch

# Выберите модель (например, Stable Diffusion)
model_id = "runwayml/stable-diffusion-v1-5"

# Загрузка модели
pipe = DiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)

# Если у вас есть GPU, переместите

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

...

unet/config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

tokenizer/vocab.json:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

vae/config.json:   0%|          | 0.00/547 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [ ]:
# Если у вас есть GPU, переместите модель на него
if torch.cuda.is_available():
    pipe = pipe.to("cuda")

# Генерация изображения по текстовому описанию
prompt = "A beautiful sunset over the mountains"  # Ваше описание
image = pipe(prompt).images[0]

# Сохранение изображения
image.save("output_image.png")

  0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
# Установка зависимостей (запустите этот блок один раз)
!pip install diffusers transformers accelerate datasets torch torchvision triton

In [ ]:
import torch
from diffusers import DiffusionPipeline, DPMSolverMultistepScheduler, DDPMScheduler, UNet2DConditionModel
from transformers import CLIPTextModel, CLIPTokenizer
from datasets import load_dataset
from torch.utils.data import DataLoader
import os

# Шаг 1: Загрузка базовой модели
model_id = "runwayml/stable-diffusion-v1-5"
pipe = DiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)

if torch.cuda.is_available():
    pipe = pipe.to("cuda")

# Замена планировщика для ускорения обучения
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)

# Шаг 2: Подготовка датасета
dataset = load_dataset("huggan/fashion-gen", split="train")
dataset = dataset.shuffle(seed=42).select(range(100))  # Используем только 100 примеров для демонстрации

# Предобработка данных
def preprocess_data(examples):
    images = [example["image"] for example in examples]
    texts = [example["text"] for example in examples]

    tokenizer = CLIPTokenizer.from_pretrained(model_id)
    input_ids = tokenizer(texts, padding="max_length", truncation=True, max_length=77, return_tensors="pt").input_ids

    # Преобразование изображений в тензоры
    from torchvision import transforms
    transform = transforms.Compose([
        transforms.Resize((512, 512)),
        transforms.ToTensor(),
        transforms.Normalize([0.5], [0.5]),
    ])
    pixel_values = [transform(image) for image in images]

    return {"input_ids": input_ids, "pixel_values": torch.stack(pixel_values)}

dataset.set_transform(preprocess_data)

# Создание даталоадера
train_dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

# Шаг 3: Настройка параметров обучения
batch_size = 4
num_epochs = 2
gradient_accumulation_steps = 1
learning_rate = 1e-5
max_train_steps = 500

# Заморозка весов основной части модели
for param in pipe.unet.parameters():
    param.requires_grad = False

# Разморозка последних слоев для fine-tuning
for param in pipe.unet.up_blocks[-1].parameters():
    param.requires_grad = True

# Определение оптимизатора
optimizer = torch.optim.AdamW(
    pipe.unet.parameters(),
    lr=learning_rate,
)

# Шаг 4: Обучение модели
losses = []
pipe.train()
for epoch in range(num_epochs):
    for step, batch in enumerate(train_dataloader):
        optimizer.zero_grad()

        # Генерация шума и предсказание
        noise = torch.randn_like(batch["pixel_values"]).to(pipe.device)
        bsz = batch["pixel_values"].shape[0]
        timesteps = torch.randint(0, pipe.scheduler.num_train_timesteps, (bsz,), device=pipe.device).long()

        noisy_images = pipe.scheduler.add_noise(batch["pixel_values"].to(pipe.device), noise, timesteps)
        noise_pred = pipe.unet(noisy_images, timesteps, batch["input_ids"].to(pipe.device)).sample

        # Вычисление потерь
        loss = torch.nn.functional.mse_loss(noise_pred, noise)
        loss.backward()
        optimizer.step()

        losses.append(loss.item())
        if step % 10 == 0:
            print(f"Epoch {epoch}, Step {step}, Loss: {loss.item()}")

# Сохранение обновленной модели
output_dir = "my_fine_tuned_model"
os.makedirs(output_dir, exist_ok=True)
pipe.save_pretrained(output_dir)

# Шаг 5: Генерация изображений
# Загрузка обновленной модели
pipe = DiffusionPipeline.from_pretrained(output_dir, torch_dtype=torch.float16)

if torch.cuda.is_available():
    pipe = pipe.to("cuda")

# Генерация изображения
prompt = "A beautiful sunset over the mountains"
image = pipe(prompt).images[0]

# Сохранение изображения
image.save("generated_image.png")